In [ ]:
%matplotlib inline
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from matplotlib import pyplot as plt
from pylab import *  # for figsize
import matplotlib.dates as mdates
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
import seaborn as sns
sns.set()
sns.set(style="ticks")

import sklearn as sl
import sklearn.linear_model
import sklearn.datasets
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LinearRegression
import io
import requests
import os
import tarfile
from six.moves import urllib
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
#from future_encoders import ColumnTransformer
from sklearn.model_selection import StratifiedShuffleSplit as SSS
from sklearn.preprocessing import Normalizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Table of Content

In [ ]:
%%javascript
$('<div id="toc"></div>').css({position: 'fixed', top: '120px', left: 0}).appendTo(document.body);
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js');

## Cleaning

### Loading the data

It is important to set the decimal argument, when commatas are used

In [ ]:
existing = pd.read_csv("existing.csv", sep = ";", decimal = ",") 
new = pd.read_csv("new.csv", sep = ";", decimal = ",")

In [ ]:
#new.head()

### Create whole dataframe

In [ ]:
frames = [existing, new]
whole = pd.concat(frames)

### Summary statistics

In [ ]:
#whole4.head()

In [ ]:
#list(whole3)

In [ ]:
#existing.info()

Best_seller_rank has 15 missing or incorrect values, Width has one. 

In [ ]:
#summary = whole4.describe()
#summary = summary.transpose()

In [ ]:
#summary #whole4

Here there are not all attributes displayed

In [ ]:
#whole2.dtypes

### Convert object columns to float

create dataframe without Product_type to do conversion

In [ ]:
whole2 = whole

In [ ]:
whole2 = whole2.drop(columns=["Product_type"])

Conversion

In [ ]:
cols = whole2.select_dtypes(exclude=['int64']).columns

whole2[cols] = whole2[cols].apply(pd.to_numeric, downcast='float', errors='coerce')

Add Product_type column again

In [ ]:
whole2["Product_type"] = whole["Product_type"]

### Look at NANs

In [ ]:
#whole2[whole2.isnull().any(axis=1)]

In [ ]:
#whole2[whole2.isnull().any(axis=1)]["Best_seller_rank"]

We can see that Best_seller_rank has 15 NANs and Width has one

### Replace NANs with median in Width

Add an imputer here

In [ ]:
whole2["Width"].fillna(whole2["Width"].median(),inplace=True);
whole2["Width"].median()

In [ ]:
whole2["Width"][97]

### Delete rows which have NAN in Best_seller_rank

In [ ]:
whole2 = whole2.dropna()

## Attribute Selection

Looking at the summary of whole2 again, Best seller rank has outliers, and since it is probably highly correlated with Volume, we can safely ignore it and drop it.

In [ ]:
whole3 = whole2.drop(columns=["Best_seller_rank"])

### scatter-plot matrix & SHOW-ZONE

In [ ]:
#cols = ["5Stars",'4Stars',
# '3Stars',
# '2Stars',
# '1Stars',
#       ]#'Positive_service_review',
# #'Negative_service_review',
# #'Would_consumer_recomend__product',]
#
#sns.pairplot(whole3[cols])

In [ ]:
attributes = ["5Stars","Positive_service_review", "Volume"]
#scatter_matrix(training1[attributes], figsize=(30, 20))

In [ ]:
training3.plot(kind="scatter", x="5Stars", y="Volume",
             alpha=0.5, figsize=(20,10))
plt.axis([0,1000,0,1000])

In [ ]:
#training1.plot(kind="scatter", x="Positive_service_review", y="Volume",
#             alpha=0.5, figsize=(20,10))
#plt.axis([0,400#,0,4000])

In [ ]:
bins1 = [10,30,45,60,75,90,105,500]
sns.distplot(pred_set["5Stars"], bins = bins1) #use prediction set and training set (1.o) to compare
plt.xlim(0,110)

### Correlation matrices

We will evaluate the correlations of the whole dataset (training and test set included), which is a "quick-and-dirty-solution". 

Correlation matrix star-reviews

In [ ]:
cols =["5Stars",'4Stars',
'3Stars',
'2Stars',
'1Stars', "Volume"]
#whole3[cols].corr()

We will just keep 5-stars

Correlation matrix 5-stars, negative-, positive-reviews

In [ ]:
cols=['Positive_service_review',
 'Negative_service_review',"5Stars", "Volume"]

#whole3[cols].corr()

We are going to keep positive reviews, it is correlated with volume, well enough, and not too correlated with 5-stars

Another correlation matrix

In [ ]:
cols = ['Would_consumer_recomend__product',
 'Weigth',
 'Depth',
 'Width',
 'Heigth',
 'Profit_margin',"5Stars",'Positive_service_review', "Volume"]

#whole3[cols].corr()

The newly included attributes are not strongly correlated with the Volume. We won't keep them in.

In [ ]:
cols = ['Profit_margin',
 'Volume',
 'Relative_Price',
 'Environment_Impact',
 'Durability_standard',
 'Product_type',"5Stars",'Positive_service_review']

#whole3[cols].corr()

None of the added attributes is highly enough correlated with the Volume.

### Conclusion of Selection

In this first approach of selecting a combination of attributes, we focused on the correlation matrices, i.e. we looked for linear relationsships. Thus we will, as a first approximation, try to get good prediction results by applying a linear model.
We chose 5-stars and positive service reviews as only relevent attributes.

In [ ]:
keeps = ["5Stars",'Positive_service_review',"Volume"]
whole4 = whole3[keeps]
#whole4.head()

## Preprocessing

The operations before only targeted rows/columns of the training set (1.o.), now we have to remove outliers (before normalizing later). The removal should naturally only be applied to the training set (2.o.). 
Therefore we have to split into training set (1.o.) and prediction set and then again the training set (1.o.) into training set (2.o.) and test set. 

### Splitting into training-set (1 order) and prediction-set

find index, where prediction set begins by using summary statistics

In [ ]:
whole4.iloc[228]

In [ ]:
#whole6.index[230]

Split 

In [ ]:
pred_set = whole4.iloc[range(230,len(whole4),1)]
#pred_set.head()

In [ ]:
training1 = whole4.iloc[range(0,229,1)]
#training1.info()

In [ ]:
training1.reset_index(drop=True, inplace= True)
#training1.tail()

### Determine thresholds

It is important to get a graphical view of the chosen attributes and the dependent variable. For this we go back to the  scatter-plot show zone, and display a scatter plot matrix. 

The relationsship between the positive service reviews does not seem linear, interestingly, because the correlation was not bad.

From it we see that the relationsship between 5Stars and volume is clearly linear only after a certain amount of 5-star-reviews is reached. 
To determine this value, we take a look at the histogram (in the show-zone).

From the plot above we conclude that we set the minimum limit of 5-star reviews, which qualifies examples for our predictive model, to 20.

### Set thresholds

In [ ]:
training2 = training1.loc[training1["5Stars"] > 10]
#training2.info()

In [ ]:
pred_set2 = pred_set.loc[pred_set["5Stars"] > 10]

In [ ]:
#pred_set3 = pred_set2.loc[pred_set2["5Stars"] < 90]

In [ ]:
pred_set2.reset_index(drop=True, inplace=True)
pred_set2.info()

In [ ]:
pred_set3 = pred_set2.drop(columns=["Positive_service_review", "Volume"])

### Look at histograms for Str.Spl.

In the show-zone.

### Bin attributes for Str.Spl.

In [ ]:
training2['5Stars_cat'] = np.ceil(training2['5Stars'].values / 80)
training2['5Stars_cat'].where(training2['5Stars_cat'] < 5, 5.0, inplace=True)
training2.reset_index(drop=True, inplace= True)

In [ ]:
#training2["5Stars_cat"].value_counts()
#training2.info()

### Remove outliers & drop NANs

In [ ]:
#training3 = training2[np.abs(training2-training2.mean()) <= (3*training2.std())]

In [ ]:
#train_set["Positive_service_review"].sort_values(ascending=False).head()

In [ ]:
#whole4["Positive_service_review"].sort_values(ascending=False).head()

In [ ]:
#wholeD["Volume"].sort_values(ascending=False).head()

The example with index 49 is the maximum value of all 3 attributes. We will drop it. 

In [ ]:
training3 = training2[training2.Volume < training2.Volume.quantile(.95)]
training3 = training3.dropna()
training3.reset_index(drop=True, inplace= True)
#wholeB = wholeA[wholeA["5Stars"] < wholeA["5Stars"].quantile(.95)]
#wholeD = wholeB[wholeB["Positive_service_review"] < wholeB["Positive_service_review"].quantile(.95)]
#training3.info()

In [ ]:
training3.columns = ["5_stars", "positive_service_review", "volume", "5_stars_cats"]

### Normalization (not used)

In [ ]:
#keeps = ["5Stars", "Positive_service_review", "Volume"] # do not normalize 5stars_cat and ????dependent variable????
#training4 = training3[keeps]

In [ ]:
#training4.info()

In [ ]:
#test_set2.head(15)

In [ ]:
#x = training4.values #returns a numpy array
#min_max_scaler = preprocessing.MinMaxScaler()
#x_scaled = min_max_scaler.fit_transform(x)
#training5 = pd.DataFrame(x_scaled)#

drop 5Stars_cat & Renaming columns

In [ ]:
#training6 = pd.concat([training5, training3["5Stars_cat"]], axis=1)

In [ ]:
#training3.info()

### Stratified Splitting 

Since the training set of first order is really quite small (229 examples), we have to think about making the size of the test set not too small. So we will use a 60/40 split. 
In addition this small training size means stratifying is important.
We should consider stratifying with respect to the independent variables.

We have to look at the distribution of the two independent variables to discretize it into stratas for the stratified sampling. 

In [ ]:
Ssplit = SSS(n_splits=1, test_size=0.4, random_state=42)
Ssplit
for train_index, test_index in Ssplit.split(training3,training3["5_stars_cats"]):
    train_set = training3.loc[train_index]
    test_set = training3.loc[test_index]

In [ ]:
train_set.reset_index(drop=True, inplace= True)

In [ ]:
test_set.reset_index(drop=True, inplace= True)

Reset index of train_set

In [ ]:
#test_set.head()

In [ ]:
#skf = StratifiedKFold(n_splits=2, shuffle=False, random_state=26)

In [ ]:
#for train_index, test_index in skf.split(training2,training2["5Stars_cat"]):
#    train_set = training2.loc[train_index]
#    test_set = training2.loc[test_index]

### Standardization (not used)

In [ ]:
#st_sc = StandardScaler()

In [ ]:
#whole5 = st_sc.fit_transform(whole4)

In [ ]:
#whole6 = pd.DataFrame(whole5)

In [ ]:
#rename columns

## Training

### Stratified - Cross Validation & Linear Regression

In [ ]:
skf = StratifiedKFold(n_splits=3, shuffle=False, random_state=26) #48 /3 = 16 (number of obs per fold), 
lin_reg = LinearRegression()

In the following cross-validation-training and testing (for each fold there was a seperate test) we first used positive_service_reviews in addition to 5_stars, but the results, including solely 5Stars, were slightly better. 

In [ ]:
indices = {}
i = 0
for train_indices, test_indices in skf.split(train_set[["5_stars"]],train_set["5_stars_cats"]):
    i = i + 1
    
    #save the indices in dict to apply different models to test set 
    
    indices[i]=train_indices   
    lin_reg.fit(train_set.loc[train_indices, ["5_stars"]],
                train_set.loc[train_indices, ["volume"]])
    print("\n")
    print(str(i) +". fold")
    print("\n")
    
    # printing predictions on validation sets (not needed)
    
    #print(lin_reg.predict(train_set3.loc[test_indices, ["5_stars", "positive_service_review"]]))
    #print("\n")
    
    #printing R-squared on validation set
    print("R-squared on validation set")
    print(lin_reg.score(train_set.loc[test_indices,["5_stars"]],train_set.loc[test_indices,["volume"]]))
    
    #printing coefficients of regressions on different folds (not needed)
    
    #print("\n")
    #print("coefficients")
    #print(lin_reg.coef_)
    #print("\n")
    #print("intercept")
    #print(lin_reg.intercept_)
    #print("\n")
    
    # prinint R-sq. of regression models of the different folds with help of indices dictionary
    print("\n")
    print("R-squared on test set")
    print(lin_reg.fit(train_set.loc[indices[i], ["5_stars"]],
                                      train_set.loc[indices[i],["volume"]]).score(test_set[["5_stars"]],
                                                                                   test_set["volume"]))
    print("\n")
    print("RMSE on test set")
    print(sqrt(mean_squared_error(test_set["volume"], lin_reg.predict(test_set[["5_stars"]]))))
    
    #the extra brackets around  5Stars in the line above are crucial, since more than one attr. is expected
    
    #save predictions of prediction set for each fold in an extra column of the pred set3 dataframe
    pred_set3[str(i) +". fold predictions"] = lin_reg.predict(pred_set3[["5Stars"]])
        

We are constructing ranges of predictions consisting of the predictions of the first fold and the second fold, since they have the best combined R-squared values and their predictions are closest together.

## Predicting

### Notes on the final predictions

Comparing the distributions of 5-stars in the training set (1.o.) and the prediction set shows, that the one of the prediction set has a lot more values bigger than 90. This means we can only successfully predict volumes of products whose number of 5-stars varies from 10 until 90. 

The prediction set is accordingly adapted in section 4.3 

### Predictions

In [ ]:
#pred_set3.head()

### Drop & reorder columns

In [ ]:
pred_set4 = pred_set3.drop(columns = ["3. fold predictions"])

In [ ]:
cols = pred_set4.columns.tolist()
cols

In [ ]:
cols2 = [cols[0], cols[2],cols[1]]
cols2

In [ ]:
pred_set5 = pred_set4[cols2]

In [ ]:
pred_set5.columns =["5_stars", "lower_prediction_limit", "upper_prediction_limit"]

### Adding the additional column by index

In [ ]:
# set 5Stars as index for new and pred_set3

new2 = new.set_index("5Stars")
pred_set6 = pred_set5.set_index("5_stars")

In [ ]:
#new2

In [ ]:
predictions = pd.concat([new2, pred_set6], axis=1, sort=False, join_axes=[new2.index]) #join_axes to keep index

In [ ]:
#predictions.head()

In [ ]:
predictions2 = predictions.drop(columns=["Unnamed: 0"])

### Saving the data in excel file (not used)

In [ ]:
writer = pd.ExcelWriter('predictions.xlsx')
predictions2.to_excel(writer,'predictions')
#df2.to_excel(writer,'Sheet2')
writer.save()